In [67]:
import numpy as np
from scipy import linalg

In [30]:
matrix_A = np.array([[31 ,-13 ,0 ,0 ,0 ,-10 ,0 ,0 ,0 ],
[-13 ,35 ,-9 ,0 ,-11 ,0, 0, 0, 0 ],
[0 ,-9 ,31, -10, 0 ,0 ,0, 0 ,0 ],
[0 ,0 ,-10, 79, -30, 0 ,0 ,0, -9 ],
[0 ,0 ,0 ,-30, 57, -7, 0, -5 ,0 ],
[0 ,0 ,0 ,0 ,-7,47 ,-30 ,0, 0 ],
[0 ,0 ,0 ,0 ,0 ,-30, 41 ,0, 0 ],
[0 ,0 ,0 ,0 ,-5, 0, 0, 27 ,-2],
[0 ,0 ,0 ,-9 ,0 ,0 ,0, -2, 29 ]])

b = np.array([-15,27,-23,0,-20,12,-7, 7,10])

In [45]:
def JacobiIterate(A,b,epsilon):
    dimension = np.shape(A)
    for i in range(dimension[0]):
        diag = np.abs(A[i,i])
        other = np.sum(np.abs(A[i])) - diag
        if np.abs(diag) < other:
            return "error, not diagonally dominant"
        
    
    matrix_U = np.triu(A,1)
    matrix_L = np.tril(A,-1)
    
    diag = 1/np.diagonal(A)
    matrix_D_reverse = np.diag(diag)
    
    x_0 = np.zeros(dimension[1])
    
    error = 100
    k = 0 # 迭代步数
    while error > epsilon:
        # 矩阵形式的迭代
        x_1 = np.matmul(matrix_D_reverse,(b - np.matmul(matrix_L + matrix_U, x_0)))
        error = np.abs(np.max(x_1 - x_0))
        x_0 = x_1
        k = k+1

    return x_1,k

def GaussSeidelIterate(A,b,epsilon):
    dimension = np.shape(A)
    for i in range(dimension[0]):
        diag = np.abs(A[i,i])
        other = np.sum(np.abs(A[i])) - diag
        if np.abs(diag) < other:
            return "error, not diagonally dominant"
        
    
    matrix_U = np.triu(A,1)
    matrix_L = np.tril(A,-1)
    
    diag = np.diagonal(A)
    matrix_D = np.diag(diag)
    
    x_0 = np.zeros(dimension[1])
    
    k = 0 # 迭代步数
    error = 100
    while error > epsilon:
        # 回代步骤以线性方程组的求解完成
        x_1 = linalg.solve(matrix_L + matrix_D, np.matmul(-matrix_U,x_0) + b)
        error = np.abs(np.max(x_1 - x_0))
        x_0 = x_1
        k = k + 1

    return x_1, k

def SORIterate(A,b,omega,epsilon):
    dimension = np.shape(A)
    for i in range(dimension[0]):
        diag = np.abs(A[i,i])
        other = np.sum(np.abs(A[i])) - diag
        if np.abs(diag) < other:
            return "error, not diagonally dominant"
        
    
    matrix_U = np.triu(A,1)
    matrix_L = np.tril(A,-1)
    
    diag = np.diagonal(A)
    matrix_D = np.diag(diag)
    
    x_0 = np.zeros(dimension[1])
    
    k = 0 # 迭代步数
    error = 100
    while error > epsilon:
        # 回代步骤以线性方程组的求解完成
        x_1 = linalg.solve(omega * matrix_L + matrix_D, omega * b - omega * np.matmul(matrix_U,x_0) + (1 - omega) * np.matmul(matrix_D,x_0))
        error = np.abs(np.max(x_1 - x_0))
        x_0 = x_1
        k = k + 1

    return x_1, k

In [52]:
JacobiIterate(np.array([[1,2],[2,3]]),np.array([1,2]),0.000001)

print(JacobiIterate(matrix_A,b,0.0000001))
print(GaussSeidelIterate(matrix_A,b,0.0000001))
print(SORIterate(matrix_A,b,1.2,0.0000001))

(array([-0.28923376,  0.3454357 , -0.7128117 , -0.22060852, -0.43040041,
        0.15430872, -0.05782281,  0.20105389,  0.29022867]), 46)
(array([-0.28923291,  0.34543642, -0.71281135, -0.22060823, -0.43040015,
        0.15430926, -0.0578225 ,  0.20105396,  0.29022875]), 18)
(array([-0.28923391,  0.34543569, -0.71281175, -0.22060851, -0.43040043,
        0.15430874, -0.05782287,  0.20105389,  0.29022866]), 12)


认为最佳松弛因子是使得迭代步数最小的因子

In [77]:
omega_list = np.arange(0.02,2,0.02)

def SOReasy(omega):
    return SORIterate(matrix_A,b,omega,0.0000001)[1]

steps = [SOReasy(omega) for omega in omega_list]
print(steps)

[282, 139, 444, 342, 278, 234, 202, 178, 158, 143, 130, 119, 109, 101, 94, 88, 82, 77, 73, 69, 65, 61, 58, 55, 53, 50, 48, 46, 44, 42, 40, 38, 37, 35, 34, 32, 31, 30, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 19, 18, 17, 16, 15, 15, 14, 13, 12, 11, 11, 12, 14, 14, 14, 14, 16, 16, 16, 18, 19, 19, 20, 22, 23, 25, 25, 25, 29, 32, 35, 35, 38, 41, 41, 51, 54, 57, 67, 80, 93, 99, 138, 167, 235, 426, 1631, 45047, 18075, 11391, 8359]


In [78]:
print("最小迭代步数为 ",min(steps)," ，对应松弛因子 ",(steps.index(min(steps)) + 1)/50)

最小迭代步数为  11  ，对应松弛因子  1.16
